In [ ]:
#1. Desarrollar un script en python que baje desde Buenos Aires Data el JSON correspondiente a los Centros Médicos Barriales.
import requests
from pprint import pprint
from pymongo import MongoClient

In [ ]:
url = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/salud/centros-medicos-barriales/centros-medicos-barriales.geojson'
response = requests.get(url)
objeto = response.json()
#pprint(objeto)
pprint(objeto['features'])
for estacion in objeto['features']:
    pprint(estacion['properties'])

In [ ]:
#2. Insertar ese JSON en una base de datos de mongo.
cliente = MongoClient("mongodb://localhost:27017/")
bd = cliente['salud']
coleccion = bd['centros']
coleccion.remove()
coleccion.insert_many(objeto['features'])

In [ ]:
#3. Realizar un query que traiga los datos de los Centros Médicos Barrialess correspondientes al barrio de Barracas, ordenados por el campo OBJECTID, descendente.
client = MongoClient('mongodb://localhost:27017/')
filter={'properties.BARRIO': 'Barracas'}
sort=list({'properties.OBJECTID': -1}.items())
result = client['salud']['centros'].find(filter=filter,sort=sort)
for resultado in result:
   pprint(resultado)

In [ ]:
#4. Realizar un query que traiga los datos de las Centros Médicos Barriales donde dentro de las especialidades se incluya Odontología.
import re
client = MongoClient('mongodb://localhost:27017/')
filter={'properties.ESPECIALID': re.compile(r"Odonto")}
result = client['salud']['centros'].find(filter=filter)
for resultado in result:
   pprint(resultado)

In [ ]:
#5. Generar a partir de este JSON un archivo de salida de tipo CSV, para su posterior geovisualización en www.gpsvisualizer.com , como fue realizado en el módulo 1, considerando que los campos/encabezados deben ser: latitude, longitude, name, label.

client = MongoClient('mongodb://localhost:27017/')
filter={}
project={
    'properties.NOMBRE': 1, 
    'properties.Dom_mapa': 1, 
    'geometry.coordinates': 1
}
result = client['salud']['centros'].find(filter=filter, projection=project)
salida = open('centros.csv', 'w', encoding='utf-8')
salida.write('latitude,longitude,name,label\n')
for resultado in result:
   nombre = resultado['properties']['NOMBRE']
   direccion = resultado['properties']['Dom_mapa']
   lon = resultado['geometry']['coordinates'][0]
   lat = resultado['geometry']['coordinates'][1]
   salida.write(f'{lat},{lon},{direccion},{nombre}\n')

salida.close()